**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/gym_plannable.git
!{sys.executable} -m pip install git+https://github.com/michalgregor/rl_tabular.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
%matplotlib inline
from gym_plannable.env import MazeEnv
from rl_tabular import StateValueTable, ActionValueTable, collect_states
from rl_tabular.maze_env_plots import plot_action_values, plot_state_values
from rl_tabular import vtable_control, qtable_control
import numpy as np

## Dynamické programovanie: iterácia hodnotami

Ako sme sa už dozvedeli, učenie s odmenou je o nájdení stratégie, ktorá by maximalizovala dlhodobé odmeny. Vieme tiež, že jedna skupina týchto prístupov je založená na hodnotových funkciách.

*Hodnotová funkcia stavov*  $V(s)$ vyjadruje, aké dlhodobé odmeny môžeme očakávať v budúcnosti ak sa teraz nachádzame v stave $s$ a ďalej sa budeme riadiť stratégiou $\pi$ [[sutton1998]](#sutton1998):

$$
V^{\pi}(s) = \mathbb{E}_{\pi}\{ R_{t}|s_{t}=s \}
$$
Spomeňte si tiež, že ak máme k dispozícii distribučný model prostredia a priestor akcií je malý, môžeme hodnotovú funkciu stavov vypočítať pomocou metódy známej ako **iterácia hodnotami** .

Na začiatku nastavíme hodnoty všetkých stavov na nuly:



In [ ]:
vtable = StateValueTable()
plot_state_values(vtable);

Potom aplikujeme nasledujúce pravidlo, pričom iterujeme viackrát po všetkých stavoch $s$:

$$
V_{\color{red} k+1}(s) = \max_{a \in A} \sum_{s'} P_{ss'}^{a} \left[r_{ss'}^{a} + \gamma V_{\color{red} k}(s') \right].
$$kde $P_{ss'}^{a}$ je pravdepodobnosť prechodu zo stavu $s$ do stavu $s'$ po vykonaní akcie $a$; a $r^a_{ss'}$ je odmena získaná za ten istý prechod.

---
### Úloha 1: Iterácia hodnotami

**Do nasledujúcej bunky doplňte pravidlo iterácie hodnotami.** 

$$
V_{\color{red} k+1}(s) = \max_{a \in A} \sum_{s'} P_{ss'}^{a} \left[r_{ss'}^{a} + \gamma V_{\color{red} k}(s') \right].
$$
---


In [ ]:
gamma = 0.9
env = MazeEnv()
env.reset()

# we collect all reachable states
states = collect_states(env.single_plannable_state())

for it in range(15):
    print(f"Iteration {it} started.")
    
    for state in states:
        if state.is_done():
            continue
            
        legals = state.legal_actions()
        maxval = -np.inf

        for a in legals:
            val = 0

            for next_state, prob in state.all_next(a):
                r = next_state.reward()                
                
                
                
                # r, gamma, prob, vtable, next_state       
                val +=     # -----

                
                
                
            maxval = max(val, maxval)

        vtable[state] = maxval
        
    plot_state_values(vtable, states, env=env, update_display=True)

### Riadenie agenta na základe hodnotovej funkcie stavov

Ak je priestor akcií diskrétny a relatívne malý a už poznáme optimálnu hodnotovú funkciu, je ľahké z nej odvodiť optimálnu stratégiu – jednoducho iterujeme cez všetky akcie a vyberieme tú, u ktorej je najpravdepodobnejšie, že bude viesť do stavu s vysokou hodnotou.



In [ ]:
env = MazeEnv(render_mode='human', show_path=True)
vtable_control(env, vtable, max_steps=100)

### Konverzia medzi hodnotovou funkciou stavov a akcií

Ak máme k dispozícii model, hodnotovú funkciu stavov je možné konvertovať na hodnotovú funkciu akcií podľa v podstate rovnakej logiky, akú sme aplikovali pri riadení agenta podľa hodnotovej funkcie stavov.



In [ ]:
qtable = vtable.to_action_values(states)
plot_action_values(qtable, states, action_spec=env.action_spec)

Je tiež možné konvertovať hodnotovú funkciu akcií na hodnotovú funkciu stavov:



In [ ]:
vtable2 = qtable.to_state_values()
plot_state_values(vtable2, states)

### Riadenie pomocou hodnotovej funkcie akcií

Riadenie agentov pomocou hodnotovej funkcie akcií je ešte jednoduchšie než pomocou hodnotovej funkcie stavov. Keďže hodnotová funkcia akcií nám poskytuje priamo hodnoty akcií, stačí nám vybrať akciu s najvyššou hodnotou – nepotrebujeme už model, ktorý by nás informoval o možných nasledujúcich stavoch atď.



In [ ]:
env = MazeEnv(render_mode='human', show_path=True)
qtable_control(env, qtable, max_steps=100)

### References

<a id="sutton1998">[sutton1998]</a> SUTTON, R.S. - BARTO, A.G. Reinforcement Learning: An Introduction. [s.l.]: The MIT Press, 1998. ISBN 0262193981.

